**Задание 1**  
Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
Логин тестового ящика: study.ai_172@mail.ru
Пароль тестового ящика: NextPassword172

In [1]:
import json
from pprint import pprint
import time
from pymongo import MongoClient
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

with open('user_params.json') as file:
    user_params = json.loads(file.read())
    
chrome_options = Options()
chrome_options.add_argument('start-maximized')

In [2]:
# Функция добавления документов в коллекции. В качестве аргументов принимает 
# добавляемый словарь и коллекцию для добавления
def add_collection(news_dict, collection, check_name):
    for new in news_dict:
        count = 0
        if collection.count_documents({check_name: new[check_name]}) == 0:
            collection.insert_one(new)
            count += 1
    print(f'В коллекцию {collection.name} добавлено {count} документов. Всего документов в коллекции {collection.count_documents({})}')
    
# Функция очистки словаря
def clear_dict(dictionary, del_features):
    for dict_el in dictionary:
        for el in del_features:
            del dict_el[el]
    return dictionary

In [3]:
driver = webdriver.Chrome('./chromedriver.exe', options=chrome_options)
driver.get('https://mail.ru')
wait = WebDriverWait(driver, 10)

# Выбираем и заполняем поле "Имя аккаунта"
user_name = driver.find_element_by_id('mailbox:login-input')
user_name.send_keys(user_params['username'])
user_name.submit()

# Выбираем и заполняем поле пароля
user_login = driver.find_element_by_id('mailbox:password-input')
user_login.send_keys(user_params['password'])
user_login.submit()

# Выбираем первое входящее сообщение и заходим в него
first_message = wait.until(
    EC.presence_of_element_located(
        (By.XPATH,
         "//div[@class='dataset__items']//a[contains(@class, 'llc')][1]")))
driver.get(first_message.get_attribute('href'))
# Собираем требуемые данные с сообщения и переходим к следующему
# Цикл работает до последнего сообщения
mail_messages = []

while True:
    time.sleep(3)
    message = {}
    
    title = driver.find_element(By.CLASS_NAME,'thread__subject')
    from_ = driver.find_element(By.XPATH, 
                                "//div[@class='letter__author']/span")
    date = driver.find_element(By.XPATH, 
                               "//div[contains(@class, 'letter__date')]")
    content = driver.find_element(By.XPATH, "//div[contains(@class, 'cl_')]")
    
    data_id = driver.find_element(
        By.XPATH, 
        "//div[contains(@class, 'thread__letter')]").get_attribute('data-id')

    message['title'] = title.text
    message['from'] = from_.text
    message['date'] = date.text
    message['content'] = content.text
    message['data_id'] = data_id
    
    mail_messages.append(message)
        
    button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH,
             "//span[contains(@class, 'button2_arrow-down')]")))
    
    if 'disabled' in button.get_attribute('class'):
        break
    else:
        button.click()
        
# Завершаем работу webdriver
driver.quit()

In [5]:
# Подключаемся к MongoDB
client = MongoClient('127.0.0.1', 27017)

# Cоздаем указатель к нашей базе данных
db = client['mail_messages']

# Создаем коллекцию
mail_messages_db = db.mail_messages

# Добавляем документы в соответствующие коллкции
add_collection(mail_messages, mail_messages_db, 'data_id')

В коллекцию mail_messages добавлено 1 документов. Всего документов в коллекции 124


**Задание 2**  
Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары

In [6]:
# Заходим на страницу mvideo.ru
driver = webdriver.Chrome('./chromedriver.exe', options=chrome_options)
driver.get('https://www.mvideo.ru/')
wait = WebDriverWait(driver, 10)

# Переходим к карусели "Хиты продаж"
hits_block = wait.until(
    EC.presence_of_element_located(
                        (By.XPATH,
                         "//div[contains(text(),'Хиты продаж')]/../../..")))
action = ActionChains(driver)
action.move_to_element(hits_block).perform()

# Определяем кнопку прокрутки карусели
button = wait.until(
        EC.element_to_be_clickable(
            (By.XPATH,
             "//div[contains(text() ,'Хиты продаж')]/../../..//a[contains(@class, 'next-btn')]")))

# Прогружаем весь список карусели "Хиты продаж"
while 'disabled' not in button.get_attribute('class'):
    button.click()
    
# Соберем все необходимые элементы блока "Хиты продаж" 
# и выберем необходимую информацию
goods = driver.find_elements(
    By.XPATH, 
    "//div[contains(text() ,'Хиты продаж')]/../../..//li//a[@class='sel-product-tile-title']")
goods_info = [el.get_attribute("data-product-info") for el in goods]

# Завершаем работу webdriver
driver.quit()

In [8]:
# Подготовим полученные данные 
# Отчистка от лишних символов
goods_info = [el.replace('\t\t\t\t\t', '').replace('\n', '') for el in goods_info]

# Преобразуем в словарь
goods_dict = [json.loads(el) for el in goods_info]
pprint(goods_dict[0])

{'Location': 'block5260655',
 'eventPosition': 5,
 'productCategoryId': 'cat2_cis_0000000062',
 'productCategoryName': 'Электрочайники',
 'productGroupId': 'cat1_cis_0000000003',
 'productId': '20062760',
 'productName': 'Электрочайник Moulinex BY730132',
 'productPriceLocal': '1990.00',
 'productVendorName': 'Moulinex'}


In [9]:
# Удалим лишние значения в словарях
del_features = ['Location', 'eventPosition', 
            'productCategoryId', 'productGroupId']
hit_goods_info = clear_dict(goods_dict, del_features)
pprint(hit_goods_info[0])

{'productCategoryName': 'Электрочайники',
 'productId': '20062760',
 'productName': 'Электрочайник Moulinex BY730132',
 'productPriceLocal': '1990.00',
 'productVendorName': 'Moulinex'}


In [10]:
# Подключаемся к MongoDB
client = MongoClient('127.0.0.1', 27017)

# Cоздаем указатель к нашей базе данных
db = client['mvideo_hits']

# Создаем коллекцию
mvideo_hits_db = db.mvideo_hits

# Добавляем документы в соответствующие коллкции
add_collection(hit_goods_info, mvideo_hits_db, 'productId')

В коллекцию mvideo_hits добавлено 1 документов. Всего документов в коллекции 16
